# 🧠 Phase 3 Design: Dynamic MNIST Digit Recognition 💡

**🎯 Goal:** Refine the design and implementation plan for Phase 3: Recognizing a **dynamic number** of digits (0 to `max_digits`) placed **randomly** within a larger image canvas, including identifying **empty areas**, using **Approach A (Grid Classification)**.

## 1. Recap: Problem Definition & Chosen Approach

*   **Input:** Image (e.g., 64x64) containing 0 to N randomly placed MNIST digits.
*   **Output:** Classification for *each cell* in a predefined grid (e.g., 8x8=64 cells), predicting either a digit (0-9) or an 'empty' class (10).
*   **Challenges:** Variable object count, localization implicitly tied to grid cells, potential class imbalance ('empty' vs. digits).
*   **Chosen Path:** Approach A - Treat it as a dense, per-cell classification problem using the ViT encoder output.

## 2. Refined Implementation Plan (Approach A) ✍️

### 2.1. Dataset Generation (`dataset.py`)

*   **Parameters (from `config.yaml`):**
    *   `image_size_phase3` (e.g., 64)
    *   `patch_size_phase3` (e.g., 8 - **Note:** patch size now determines the grid cell size)
    *   `max_digits_phase3` (e.g., 5)
    *   `num_classes_phase3` (should be 11: 0-9 + 10 for 'empty')
*   **`generate_dynamic_digit_image` Function:**
    1.  Create empty canvas (e.g., `torch.zeros((1, image_size_phase3, image_size_phase3))`).
    2.  Determine number of digits `k = random.randint(0, max_digits_phase3)`.
    3.  Sample `k` MNIST digits (image tensors + labels).
    4.  For each sampled digit:
        *   Find a random, non-overlapping top-left coordinate `(y, x)` on the canvas to place the 28x28 digit.
        *   *Placement Strategy:* Ensure digits don't heavily overlap? Or allow overlap?
        *   Paste the digit onto the canvas tensor.
        *   Store the digit's label and its bounding box `(y_min, x_min, y_max, x_max)` on the canvas.
    5.  Create the target label grid `target_grid = torch.full((grid_h, grid_w), fill_value=10, dtype=torch.long)` where `grid_h = grid_w = image_size_phase3 // patch_size_phase3` (e.g., 8x8). Label 10 represents 'empty'.
    6.  **Mapping Digits to Grid Cells (Crucial Decision):**
        *   *Option 1 (Center Point):* For each placed digit, calculate its center `(cy, cx)`. Determine which grid cell `(cell_y, cell_x)` contains this center. Assign `target_grid[cell_y, cell_x] = digit_label`.
        *   *Option 2 (Majority Overlap - More Complex):* For each grid cell, determine which digit (if any) covers the *majority* of that cell's area. Assign the label accordingly.
        *   **Recommendation:** Start with **Option 1 (Center Point)** for simplicity.
    7.  Return the `canvas_image` tensor and the `target_grid` tensor (flattened to `(NumCells)`? Or keep as `(GridH, GridW)`? Let's keep grid shape for now: `(GridH, GridW)`).
*   **`MNISTDynamicDataset(Dataset)` Class:**
    *   Takes base MNIST dataset, desired `length`, image/patch sizes.
    *   `__getitem__` calls `generate_dynamic_digit_image` and returns the generated `(canvas_image, target_grid)`.

### 2.2. Model Adaptation (`model.py`)

*   **`VisionTransformer.__init__`:**
    *   Instantiate with `img_size` and `patch_size` relevant to Phase 3 (e.g., 64, 8).
    *   The number of patches `num_patches` will correspond to the number of grid cells (e.g., 64).
    *   Set `num_classes` to 11 (0-9 + empty).
    *   Set `num_outputs` to `num_patches` (e.g., 64). **Each patch output will predict the class for its corresponding grid cell.**
    *   Final MLP `head` output size: `num_patches * num_classes` (e.g., `64 * 11`).
*   **`VisionTransformer.forward`:**
    *   **Change Output Source:** Instead of taking only the CLS token (`x[:, 0]`), take the outputs for **all patch tokens** *after* the final encoder block and final LayerNorm: `patch_outputs = self.norm(x[:, 1:])` (shape `(B, NumPatches, EmbedDim)`).
    *   **Modify Head Application:** Apply the *same* MLP head independently to *each* patch's output embedding. This requires the head to be `nn.Linear(embed_dim, num_classes)`. *Correction to previous plan: Head size is just `num_classes` (11), not `num_patches * num_classes`. The sequence dimension handles the different patches.*
    *   **Revised `forward` end:**
        ```python
        # Inside forward method...
        # x shape after encoders: (B, NumPatches+1, EmbedDim)
        
        # Apply final LayerNorm to patch tokens (excluding CLS token)
        patch_outputs = self.norm(x[:, 1:]) # Shape: (B, NumPatches, EmbedDim)
        
        # Apply head to each patch embedding
        # Head should be nn.Linear(embed_dim, num_classes=11)
        logits = self.head(patch_outputs) # Shape: (B, NumPatches, NumClasses=11)
        
        # No final reshape needed if loss handles (B, N, C)
        return logits
        ```
*   **`VisionTransformer.__init__` Head Correction:**
    ```python
    # Inside __init__
    self.head = nn.Linear(embed_dim, num_classes) # Output is just 11 classes per patch
    ```
*   **Note:** This removes the dependency on the CLS token for classification in this phase, treating each patch output as responsible for predicting its corresponding grid cell class.

### 2.3. Trainer Adaptation (`trainer.py`)

*   **Add `elif phase == 3:` in `train_epoch` and `evaluate_model`:**
*   **Loss Calculation:**
    *   Model output `outputs` shape: `(B, NumPatches, 11)`.
    *   Target labels `labels` shape: `(B, GridH, GridW)` or flattened `(B, NumPatches)`. Let's assume flattened `(B, NumPatches)` for simplicity.
    *   Reshape `outputs` -> `(B * NumPatches, 11)`.
    *   Reshape `labels` -> `(B * NumPatches)`.
    *   Calculate `nn.CrossEntropyLoss`. 
    *   **Class Weights:** Define `class_weights = torch.tensor([...], device=device)`. A starting point could be weight 1.0 for digits 0-9 and a lower weight (e.g., 0.1-0.5) for the 'empty' class (index 10), depending on how frequent it is. Pass `weight=class_weights` to `nn.CrossEntropyLoss(weight=class_weights)`.
*   **Accuracy Calculation (`evaluate_model`):**
    *   Get predictions `predicted = torch.argmax(outputs.data, dim=-1)` (shape `(B, NumPatches)`).
    *   Compare `predicted == labels.view(batch_size, -1)`.
    *   Sum correct predictions. Total samples = `labels.numel()`.
    *   **Optional:** Calculate separate accuracy for digit predictions vs. empty predictions.

### 2.4. Main Script (`train_mnist_vit.py`) & Config (`config.yaml`)

*   Add `elif args.phase == 3:` to load `MNISTDynamicDataset`.
*   Ensure `VisionTransformer` is instantiated with Phase 3 parameters from `config.yaml` (image size, patch size, num classes=11, num_outputs=NumPatches - *wait, model change removed num_outputs*).
*   Pass `phase=3` to `train_model`.
*   Update `config.yaml` with finalized Phase 3 parameters.

## 3. Addressing Open Questions (Initial Thoughts) 🤔

*   **Mapping Digits to Cells:** Start with the **center point** method. If a digit's center falls within a patch/cell, label that cell with the digit class. This is simpler than area overlap.
*   **Class Imbalance:** Use the `weight` argument in `nn.CrossEntropyLoss`. Start by giving the 'empty' class (index 10) a lower weight (e.g., 0.2) and digits 0-9 weight 1.0. Tune based on validation performance (is it predicting 'empty' too often?).
*   **Positional Embeddings:** Stick with the standard learned 1D positional embeddings initially (`PatchEmbedding` handles this). They encode absolute position. If performance is poor spatially, 2D embeddings (like in original ViT paper appendix) or relative embeddings could be explored later.
*   **CLS Token vs. Per-Patch:** The revised plan uses **per-patch outputs** after the final encoder block, which seems more natural for a grid classification task than relying solely on the CLS token. We take `x[:, 1:]` (all patch tokens) after the encoder, apply the final `LayerNorm`, and then apply the *same* classification head (`Linear(embed_dim, 11)`) to each patch token's embedding.

## 4. Refined Next Steps 🚀

1.  Implement `generate_dynamic_digit_image` and `MNISTDynamicDataset` in `dataset.py`. Test generation locally (maybe in *this* notebook initially).
2.  Modify `model.py` (`VisionTransformer.forward`) to use per-patch outputs and adjust the head layer size.
3.  Modify `trainer.py` (`train_epoch`, `evaluate_model`) to handle Phase 3 reshaping and loss calculation (including basic class weighting).
4.  Update `config.yaml` for Phase 3. Modify `train_mnist_vit.py` to load Phase 3 data and pass `phase=3`.
5.  Run initial Phase 3 training runs. Debug and analyze results on W&B, focusing on both digit and 'empty' cell classification accuracy. Iterate on class weights, hyperparameters.